In [5]:
import os
import sys

from tqdm import tqdm

import torch
from unsloth import FastLanguageModel, FastVisionModel


print("cuda is available:",torch.cuda.is_available())
print("cuda device count:", torch.cuda.device_count())

sys.path.append('../src')

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
cuda is available: True
cuda device count: 2


In [6]:
model_name = 'unsloth/gpt-oss-20b-unsloth-bnb-4bit'

max_seq_length = 200000 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastVisionModel.from_pretrained(
        model_name = model_name,
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
        cache_dir = "../models",
        # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
    )

model = FastLanguageModel.get_peft_model(
    model,
    # Specify the existing model

    r=16,  # Choose any positive number! Recommended values include 8, 16, 32, 64, 128, etc.
    # Rank parameter for LoRA. The smaller this value, the fewer parameters will be modified.

    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj",],
    # Specify the modules to which LoRA will be applied

    lora_alpha=16,
    # Alpha parameter for LoRA. This value determines the strength of the applied LoRA.

    lora_dropout=0,  # Currently, only supports dropout = 0
    # Dropout rate for LoRA. Currently supports only 0.

    bias="none",  # Currently, only supports bias = "none"
    # Bias usage setting. Currently supports only the setting without bias.

    use_gradient_checkpointing='unsloth',
    # Whether to use gradient checkpointing to improve memory efficiency

    random_state=3407,
    # Seed value for random number generation

    max_seq_length=max_seq_length,
    # Set the maximum sequence length
)

==((====))==  Unsloth 2025.8.6: Fast Gpt_Oss patching. Transformers: 4.55.2.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 2. Max memory: 23.988 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gpt_Oss does not support SDPA - switching to eager!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Unsloth: Making `model.base_model.model.model` require gradients


In [7]:
def extract_tag(response: str, start_tag: str, end_tag: str) -> str:
    """Extract text between specified tags"""
    start_idx = response.find(start_tag)
    end_idx = response.find(end_tag, start_idx + len(start_tag))
    
    if start_idx != -1 and end_idx != -1:
        return end_idx+len(end_tag), response[start_idx + len(start_tag):end_idx].strip()
    return 0, ""

In [8]:
import pickle

N_BINS = 10
TOP_K = 5
NOISE_MODE = 'noisySignal'
file_name = '../../data/own/unlabeled_10k/test/noisySignal/4ASK_-5.57dB__076_20250127_145624.npy'
with open(f'../data/own/unlabeled_10k/test_{NOISE_MODE}_{N_BINS}_{TOP_K}_data.pkl', 'rb') as f:
    noisy_data = pickle.load(f)

noisy_data.keys()

dict_keys(['signal_paths', 'signals', 'stats', 'discret_stats', 'labels', 'snrs', 'prompts', 'discret_prompts', 'old_prompts', 'old_discret_prompts', 'feature_names', 'scaler', 'discretizers', 'num_samples', 'num_features', '#classes', '#snr', 'k-top'])

In [9]:
'data/own/unlabeled_10k/test/noisySignal/4ASK_-0.17dB__081_20250127_164342.npy'

'data/own/unlabeled_10k/test/noisySignal/4ASK_-0.17dB__081_20250127_164342.npy'

In [10]:
signal_path = os.path.abspath(noisy_data['signal_paths'][0][3:])
image_filename = os.path.basename(signal_path)[:-4]+'.png'
image_filename

signal_folder = os.path.dirname(signal_path)

image_path = os.path.join(os.path.dirname(signal_folder),'noisyImg' if os.path.basename(signal_folder) == 'noisySignal' else 'noiseLessImg',image_filename)
os.path.exists(image_path)

False

In [5]:
image = Image.open(image_path)
type(image)

NameError: name 'Image' is not defined

In [8]:
noisy_data['discret_prompts'][idx]

'**ROLE:**\nYou are an expert AI signal classifier specializing in wireless communication modulation schemes.\n\n**OBJECTIVE:**\nYour task is to classify the modulation scheme of a wireless signal based on its provided `KBinsDiscretizer` statistical features. You will use your domain knowledge of signal processing to interpret the features and select the correct classification from a predefined list.\n\n**CONTEXT:**\nThe classification is based on the principle that moments and cumulants of a signal\'s I/Q components create a unique feature set. For instance, the fourth-order cumulant is a powerful differentiator between modulation formats like BPSK, QPSK, and various QAM schemes. The provided statistics are the features extracted from a signal after being processed by a `KBinsDiscretizer`, which quantifies the signal\'s distribution. Your analysis should map these statistical features to the most likely modulation scheme.\n\n**Signal Statistics:** skewness: (G, E), kurtosis: (G, F), m

In [13]:
from PIL import Image
from unsloth.chat_templates import get_chat_template

# FastVisionModel.for_inference(model)  # Enable for inference!


# tokenizer = get_chat_template(
#     tokenizer,
#     chat_template = "gemma",
# )

file_name = '../../data/own/unlabeled_10k/test/noisySignal/4ASK_-5.57dB__076_20250127_145624.npy'
idx = 0 #noisy_data['signal_paths'].index(file_name)

signal_path = os.path.abspath(noisy_data['signal_paths'][idx][3:])
image_filename = os.path.basename(signal_path)[:-4]+'.png'
signal_folder = os.path.dirname(signal_path)
# image_path = os.path.join(os.path.dirname(signal_folder),'noisyImg' if os.path.basename(signal_folder) == 'noisySignal' else 'noiseLessImg',image_filename)

# image = Image.open(image_path).convert("RGB")

messages = [{
    "role": "user",
    "content": noisy_data['discret_prompts'][idx]
}]

# text = tokenizer.apply_chat_template(
#     messages,
#     add_generation_prompt = True, # Must add for generation
# )

# inputs = tokenizer(
#     # image,
#     text,
#     add_special_tokens=False,
#     return_tensors="pt",
# ).to("cuda")

text = noisy_data['discret_prompts'][idx]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "low", # **NEW!** Set reasoning effort to low, medium or high
).to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens = 2000, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    use_cache=True,
)
raw_decoded = tokenizer.batch_decode(outputs)
# image_end_idx, image_text = extract_tag(raw_decoded[0], "<start_of_image>", "<end_of_image>")
raw_response = raw_decoded[0][len(text):]

end_idx, reasoning = extract_tag(raw_response, "<|message|>", "<|message|>")

result = {
        'filename': os.path.basename(noisy_data['signal_paths'][idx]),
        'try': 0,
        "prompt": noisy_data['discret_prompts'][idx],
        "raw_response": raw_response,
        "true_label": noisy_data['labels'][idx],
        "reasoning": reasoning,
        "response_label": raw_response[end_idx:]
                    }

result

{'filename': '16PAM_-0.08dB__058_20250127_145951.npy',
 'try': 0,
 'prompt': '**ROLE:**\nYou are an expert AI signal classifier specializing in wireless communication modulation schemes.\n\n**OBJECTIVE:**\nYour task is to classify the modulation scheme of a wireless signal based on its provided `KBinsDiscretizer` statistical features. You will use your domain knowledge of signal processing to interpret the features and select the correct classification from a predefined list.\n\n**CONTEXT:**\nThe classification is based on the principle that moments and cumulants of a signal\'s I/Q components create a unique feature set. For instance, the fourth-order cumulant is a powerful differentiator between modulation formats like BPSK, QPSK, and various QAM schemes. The provided statistics are the features extracted from a signal after being processed by a `KBinsDiscretizer`, which quantifies the signal\'s distribution. Your analysis should map these statistical features to the most likely modul

In [20]:
start_idx = raw_response.find("<|message|>")
end_idx = raw_response[start_idx+len("<|message|>"):].find("<|message|>")
    
if start_idx != -1 and end_idx != -1:
        end_idx += start_idx +len("<|message|>")
        reasoning = raw_response[start_idx + len("<|message|>"):end_idx].strip()
reasoning

"We need choose based on stats. Pattern similar to earlier: skewness (G,E) kurtosis (H,G). earlier example with (G,E),(E,G) gave 4ASK. This one skewness (G,E) kurtosis (H,G) maybe similar but H instead of E maybe indicates different. Hard. Looking at pattern: first example had (G,E),(G,E) gave OQPSK. second (G,E),(E,G) gave 4ASK. third (I,E),(E,H) gave 4PAM. fourth (F,E),(F,G) gave 8ASK. fifth (G,E),(F,F) gave DQPSK.\n\nNow we have (G,E),(H,G). Which option hasn't appeared? maybe OQPSK? But H appears in kurtosis not E. maybe classification corresponds to 4ASK? uncertain.\n\nI think pattern: first value of skewness determines? Not.\n\nProbably choose 4ASK.\n\nLet's output.<|end|><|start|>assistant<|channel|>final"

In [ ]:
image_end_idx, image_text = extract_tag(raw_decoded[0], "<start_of_image>", "<end_of_image>")
raw_response = raw_decoded[0][image_end_idx+len(text)-40:]


"\n<think>\nLet's analyze the given statistics to determine the most likely modulation scheme.\n\n*   **Skewness (G, E) & Kurtosis (H, G):** These values suggest a relatively symmetrical distribution but with heavier tails than a normal distribution, indicating potential phase shifts or non-Gaussian signal characteristics. The kurtosis being high (H) points towards a modulation with more distinct amplitude levels.\n\n*   **Moments (A, A, A, etc.):** The moments are mostly 'A', indicating a balanced distribution.\n\n*   **K-Statistics (F, A, E, D, etc.):** These statistics, particularly kstat\\_4 = D, are indicative of higher-order non-Gaussianity which is seen in QAM-like signals but also in ASK/PAM.\n\nConsidering the options:\n\n*   **OQPSK:** Generally has lower kurtosis and more Gaussian-like characteristics than the provided statistics.\n*   **4ASK:** Has a limited number of amplitude levels, leading to higher kurtosis and potentially some non-Gaussianity.\n*   **4PAM:** Similar t

In [15]:
extract_think_text(raw_decoded[0][len(text):])

(1421,
 "Let's analyze the provided statistics to determine the most likely modulation scheme.\n\n*   **Skewness (G, E) & Kurtosis (E, G):** These values suggest a non-Gaussian distribution, leaning towards asymmetry, which is common in higher-order modulation schemes like PAM and QAM.\n*   **Moments (0-9):** The moments generally have low values (A, B, C, etc.), indicating a relatively well-behaved distribution, but not entirely Gaussian. The varying values suggest some level of complexity in the signal.\n*   **K-statistics (1-4):** The K-statistics, particularly kstat_4 being 'D', suggest some non-Gaussian characteristics.\n*   **kstatvar_1 & kstatvar_2:** These values being relatively low suggest some level of noise or imperfect signal shaping.\n\nConsidering the options:\n\n*   **16PAM:**  16PAM would exhibit significant non-Gaussian characteristics due to the multiple amplitude levels. The provided skewness and kurtosis values fit this expectation.\n*   **8ASK/4ASK:** ASK schemes 

In [17]:
raw_decoded[0][len(text)+1421:]

'\n16PAM\n```<end_of_turn>'

In [41]:
import json

with open('../src/evaluation/discret_prompts_gemma-3-27b-it-unsloth-bnb-4bit_noisySignal_10_5_custom_responses.json', 'r') as f:
    data = json.load(f)



In [42]:
def extract_think_text(response: str) -> str:
    """Extract text between <think> tags"""
    start_tag = "<think>"
    end_tag = "</think>"
    start_idx = response.find(start_tag)
    end_idx = response.find(end_tag)
    
    if start_idx != -1 and end_idx != -1:
        return end_idx+len(end_tag), response[start_idx + len(start_tag):end_idx].strip()
    return 0, ""



In [43]:
for item in data:
    item['raw_response'] = "<thin" +  item['raw_response']
    end_idx, reasoning = extract_think_text(item['raw_response'])
    item['reasoning'] = reasoning
    item['response_label'] = item['raw_response'][end_idx:]

item

{'filename': '4ASK_3.67dB__017_20250127_164319.npy',
 'try': 2,
 'prompt': '**ROLE:**\nYou are an expert AI signal classifier specializing in wireless communication modulation schemes.\n\n**OBJECTIVE:**\nYour task is to classify the modulation scheme of a wireless signal based on its provided `KBinsDiscretizer` statistical features. You will use your domain knowledge of signal processing to interpret the features and select the correct classification from a predefined list.\n\n**CONTEXT:**\nThe classification is based on the principle that moments and cumulants of a signal\'s I/Q components create a unique feature set. For instance, the fourth-order cumulant is a powerful differentiator between modulation formats like BPSK, QPSK, and various QAM schemes. The provided statistics are the features extracted from a signal after being processed by a `KBinsDiscretizer`, which quantifies the signal\'s distribution. Your analysis should map these statistical features to the most likely modulat

In [ ]:
# with open('../src/evaluation/discret_prompts_gemma-3-27b-it-unsloth-bnb-4bit_noisySignal_10_5_custom_responses.json', 'w') as f:
#     json.dump(data, f)

In [60]:
from typing import Dict, List

def sort_results_by_prompt(results: List[Dict]) -> Dict[int, List[Dict]]:
    """Sort results by prompt and assign numeric keys"""
    sorted_results = {}
    prompts_to_id = {}  # Dictionary to map prompts to numeric IDs
    prompt_id = 0
    
    for result in results:
        prompt = result['filename']
        if prompt not in prompts_to_id:
            prompts_to_id[prompt] = prompt_id
            prompt_id += 1
        
        current_id = prompts_to_id[prompt]
        if current_id not in sorted_results:
            sorted_results[current_id] = []
        sorted_results[current_id].append(result)
    
    return sorted_results

def find_classes_in_text(text, class_names):
    found_classes = []
    for class_name in class_names:
        if class_name in text:
            found_classes.append(class_name)
    return found_classes

def clean_acc(results: Dict[int, List[Dict]], class_names):
    """Calculate accuracy"""
    correct_count = 0
    total_count = 0
    for key, value in results.items():
        true_label = value[0]['true_label']
        for v in value:
            found_classes = find_classes_in_text(v['response_label'], class_names)
            if found_classes == []:
                print(value[0]['filename'])
            elif true_label in found_classes:
                correct_count += 1
            total_count += 1

    return (correct_count, total_count, correct_count / (total_count))

def clean_up_response_label(results: List[Dict]) -> (List[Dict], List[Dict]):
    """
    Clean up the response labels in the results by extracting valid class names.
    Returns the cleaned results and a list of problematic data samples.
    """
    problematic_data = []
    for sample in results:
        if len(sample['response_label']) > 10:
            problematic_data.append(sample)
            sample['response_label'] = sample['raw_response'][-6:]
    return results, problematic_data



In [1]:
import json
import pickle

with open("../src/evaluation/discret_prompts_gemini-2.5-flash_noisySignal_20_5_gemini_responses.json", "r") as f:
    results = json.load(f)

# Load training data
with open(f'../data/own/unlabeled_10k/test_noisySignal_10_5_data.pkl', 'rb') as f:
    data = pickle.load(f)

In [2]:
print(data['discret_prompts'][0])

**ROLE:**
You are an expert AI signal classifier specializing in wireless communication modulation schemes.

**OBJECTIVE:**
Your task is to classify the modulation scheme of a wireless signal based on its provided `KBinsDiscretizer` statistical features. You will use your domain knowledge of signal processing to interpret the features and select the correct classification from a predefined list.

**CONTEXT:**
The classification is based on the principle that moments and cumulants of a signal's I/Q components create a unique feature set. For instance, the fourth-order cumulant is a powerful differentiator between modulation formats like BPSK, QPSK, and various QAM schemes. The provided statistics are the features extracted from a signal after being processed by a `KBinsDiscretizer`, which quantifies the signal's distribution. Your analysis should map these statistical features to the most likely modulation scheme.

**Signal Statistics:** snr: E, skewness: D, kurtosis: H, moment_0: A, mo

In [61]:
cleaned_results, problematic_data = clean_up_response_label(results)
sorted_results = sort_results_by_prompt(cleaned_results)
class_names = ['4ASK', '4PAM', '8ASK', '16PAM', 'CPFSK', 'DQPSK', 'GFSK', 'GMSK', 'OQPSK', 'OOK']
clean_acc(sorted_results, class_names)

OOK_-3.26dB__008_20250127_145532.npy
GFSK_-6.18dB__004_20250127_164530.npy
4PAM_5.70dB__023_20250127_145839.npy
16PAM_6.10dB__007_20250127_164724.npy
OQPSK_4.26dB__023_20250127_145700.npy
8ASK_1.23dB__099_20250127_145653.npy


(73, 200, 0.365)

In [31]:
data = {'discret_prompts': [r['prompt'] for r in results if r['try']==0], 'signal_paths': [r['filename'] for r in results if r['try']==0]}

len(data['discret_prompts']), len(data['signal_paths'])

(332, 332)

In [37]:
print(data['old_discret_prompts'][0])

### Instructions:
You are an expert quantitative analyst in wireless communication modulation.
Based on your knowledge in wireless communication modulation and the detailed signal statistics provided below, determine the modulation type.
1. DO NOT PROVIDE ANY ADDITIONAL INFORMATION OR CONTEXT.
2. No OTHER TEXT, NO BLABBER.
### Question: skewness: (G, D), kurtosis: (F, G), moment_0: (A, A), moment_1: (A, A), moment_2: (A, A), moment_3: (F, E), moment_4: (A, A), moment_5: (F, F), moment_6: (A, A), moment_7: (F, G), moment_8: (A, A), moment_9: (E, G), kstat_1: G, kstat_2: A, kstat_3: E, kstat_4: D, kstatvar_1: A, kstatvar_2: A, 
OPTIONS: [A: OOK, B: 16PAM, C: 8ASK, D: 4ASK, E: DQPSK, F: OQPSK, G: 4PAM, H: GFSK, I: CPFSK, J: GMSK].
### Answer: OOK
### Question: skewness: (G, E), kurtosis: (E, G), moment_0: (A, A), moment_1: (A, A), moment_2: (C, B), moment_3: (F, E), moment_4: (A, A), moment_5: (F, F), moment_6: (A, A), moment_7: (F, G), moment_8: (A, A), moment_9: (E, G), kstat_1: F, ksta

In [33]:
len(set(data['old_discret_prompts'])), len(set(data['old_signal_paths']))

KeyError: 'old_signal_paths'

In [ ]:
data['old_discret_prompts']]

In [25]:
data['discret_prompts'][0]

'**ROLE:**\nYou are an expert AI signal classifier specializing in wireless communication modulation schemes.\n\n**OBJECTIVE:**\nYour task is to classify the modulation scheme of a wireless signal based on its provided `KBinsDiscretizer` statistical features. You will use your domain knowledge of signal processing to interpret the features and select the correct classification from a predefined list.\n\n**CONTEXT:**\nThe classification is based on the principle that moments and cumulants of a signal\'s I/Q components create a unique feature set. For instance, the fourth-order cumulant is a powerful differentiator between modulation formats like BPSK, QPSK, and various QAM schemes. The provided statistics are the features extracted from a signal after being processed by a `KBinsDiscretizer`, which quantifies the signal\'s distribution. Your analysis should map these statistical features to the most likely modulation scheme.\n\n**Signal Statistics:** skewness: (E, E), kurtosis: (G, G), m

In [4]:
import numpy as np
from collections import defaultdict

def sample_per_label(data, per_label=20, seed=42, shuffle=True):
    """
    Sample up to `per_label` indices per label from `data['labels']`.
    Returns (sampled_data, selected_indices, rng).
    """
    rng = np.random.default_rng(seed)

    # Group indices by label
    indices_by_label = defaultdict(list)
    for i, label in enumerate(data['labels']):
        indices_by_label[label].append(i)

    # Sample from each label group
    selected_indices = []
    for _, indices in indices_by_label.items():
        num_to_sample = min(per_label, len(indices))
        sampled = rng.choice(indices, size=num_to_sample, replace=False)
        selected_indices.extend(int(i) for i in sampled)

    # Optionally shuffle
    if shuffle:
        rng.shuffle(selected_indices)

    # Build sampled_data, preserving non-list/non-array entries
    sampled_data = {}
    for key, value in data.items():
        if isinstance(value, (list, np.ndarray)):
            sampled_data[key] = [value[i] for i in selected_indices if i < len(value)]
        else:
            sampled_data[key] = value

    # Info
    print(f"Total samples selected: {len(sampled_data.get('labels', []))}")
    print(f"Sampled data keys: {list(sampled_data.keys())}")
    unique_labels, counts = np.unique(sampled_data.get('labels', []), return_counts=True)
    print("Label distribution in sampled data:")
    for label, count in zip(unique_labels, counts):
        print(f"  {label}: {count}")

    return sampled_data, selected_indices, rng

In [5]:
sampled_data_1, selected_indices_1, rng_1 = sample_per_label(data, per_label=1, seed=42, shuffle=True)

Total samples selected: 10
Sampled data keys: ['signal_paths', 'signals', 'stats', 'discret_stats', 'labels', 'snrs', 'prompts', 'discret_prompts', 'old_prompts', 'old_discret_prompts', 'feature_names', 'scaler', 'discretizers', 'num_samples', 'num_features', '#classes', '#snr', 'k-top']
Label distribution in sampled data:
  16PAM: 1
  4ASK: 1
  4PAM: 1
  8ASK: 1
  CPFSK: 1
  DQPSK: 1
  GFSK: 1
  GMSK: 1
  OOK: 1
  OQPSK: 1


In [88]:
sampled_data_2, selected_indices_2, rng_2 = sample_per_label(data, per_label=1, seed=42, shuffle=True)

Total samples selected: 10
Sampled data keys: ['signal_paths', 'signals', 'stats', 'discret_stats', 'labels', 'snrs', 'prompts', 'discret_prompts', 'old_prompts', 'old_discret_prompts', 'feature_names', 'scaler', 'discretizers', 'num_samples', 'num_features', '#classes', '#snr', 'k-top']
Label distribution in sampled data:
  16PAM: 1
  4ASK: 1
  4PAM: 1
  8ASK: 1
  CPFSK: 1
  DQPSK: 1
  GFSK: 1
  GMSK: 1
  OOK: 1
  OQPSK: 1


In [6]:
selected_indices_1

[886, 1640, 530, 1217, 1171, 1818, 1539, 687, 17, 354]

In [89]:
selected_indices_2

[886, 1640, 530, 1217, 1171, 1818, 1539, 687, 17, 354]

In [63]:
all_prompts = data['discret_prompts'][:332]
all_prompts[:5]

['**ROLE:**\nYou are an expert AI signal classifier specializing in wireless communication modulation schemes.\n\n**OBJECTIVE:**\nYour task is to classify the modulation scheme of a wireless signal based on its provided `KBinsDiscretizer` statistical features. You will use your domain knowledge of signal processing to interpret the features and select the correct classification from a predefined list.\n\n**CONTEXT:**\nThe classification is based on the principle that moments and cumulants of a signal\'s I/Q components create a unique feature set. For instance, the fourth-order cumulant is a powerful differentiator between modulation formats like BPSK, QPSK, and various QAM schemes. The provided statistics are the features extracted from a signal after being processed by a `KBinsDiscretizer`, which quantifies the signal\'s distribution. Your analysis should map these statistical features to the most likely modulation scheme.\n\n**Signal Statistics:** skewness: (G, E), kurtosis: (E, G), 

In [59]:
def get_unique_prompts(results):
    """Get unique prompts from results"""
    return list(set(result['filename'] for result in results))

len(get_unique_prompts(data))

327

In [43]:
problematic_data = []
for sample in results:
    if len(sample['response_label']) > 10:
        problematic_data.append(sample)
        sample['response_label'] = sample['raw_response'][-6:]

In [44]:
len(problematic_data)

15

In [45]:
len(problematic_data)

15

In [47]:
count/238

0.5672268907563025

In [46]:
count = 0
for sample in results:
    is_true = sample['true_label'] in sample['response_label']
    clean_response = sample['response_label'].replace('\n', '').replace('.', '').strip()
    is_true = sample['true_label'] in clean_response
    if is_true:
        count += 1
    print(sample['true_label'], clean_response, is_true)

16PAM 16PAM True
16PAM OQPSK False
16PAM 16PAM True
16PAM 16PAM True
16PAM 16PAM True
16PAM OQPSK False
16PAM 16PAM True
16PAM 16PAM True
16PAM OQPSK False
16PAM 16PAM True
16PAM 16PAM True
16PAM 4ASK False
16PAM 16PAM True
16PAM OQPSK False
16PAM OQPSK False
16PAM OQPSK False
16PAM 16PAM True
16PAM OQPSK False
16PAM 16PAM True
16PAM 16PAM True
16PAM 16PAM True
16PAM 8ASK False
16PAM 16PAM True
16PAM 16PAM True
16PAM PAM}}$ False
16PAM 8ASK False
16PAM OQPSK False
16PAM 16PAM True
16PAM 16PAM True
16PAM OQPSK False
16PAM 16PAM True
16PAM 16PAM True
16PAM 16PAM True
16PAM 16PAM True
16PAM 16PAM True
16PAM 16PAM True
16PAM OQPSK False
16PAM PAM}}$ False
16PAM 4PAM False
16PAM 16PAM True
16PAM OQPSK False
16PAM 16PAM True
16PAM 16PAM True
16PAM OQPSK False
16PAM 16PAM True
16PAM 16PAM True
16PAM 16PAM True
16PAM 16PAM True
16PAM OQPSK False
16PAM 16PAM True
16PAM 16PAM True
16PAM 16PAM True
16PAM 4ASK False
16PAM 16PAM True
16PAM 16PAM True
16PAM PAM}}$ False
16PAM 16PAM True
16PAM 16PAM 

In [10]:
with open("discret_prompts_gemini-2.5-flash_noisySignal_10_5_gemini_responses_fixed.json", "w") as f:
    json.dump(results, f, indent=4)

In [12]:
raw_response = problematic_data[0]['raw_response']
start_tag = '<think>'
end_tag = '</think>'
_, reasoning = extract_tag(raw_response, start_tag, end_tag)
reasoning

''

In [27]:
for sample in problematic_data:
    print(sample['raw_response'][-6:].strip())

16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
OQPSK
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
OQPSK
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
.
4PAM
.
4PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
.
8ASK
16PAM
16PAM
16PAM
16PAM
OQPSK
OQPSK
OQPSK
16PAM
16PAM
16PAM
16PAM
OQPSK
OQPSK
OQPSK
16PAM
OQPSK
OQPSK
16PAM
16PAM
DQPSK
16PAM
16PAM
16PAM
16PAM
16PAM
OQPSK
OQPSK
16PAM
16PAM
16PAM
16PAM
DQPSK
OQPSK
16PAM
.
GFSK
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
.
GFSK
16PAM
16PAM
.
8ASK
CPFSK
OQPSK
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PAM
16PA

In [33]:
import pickle

N_BINS = 5
TOP_K = 2
NOISE_MODE = 'noisySignal'
with open(f'../data/own/unlabeled_10k/test_{NOISE_MODE}_{N_BINS}_{TOP_K}_data.pkl', 'rb') as f:
    noisy_data = pickle.load(f)

noisy_data.keys()

dict_keys(['signal_paths', 'signals', 'stats', 'discret_stats', 'labels', 'snrs', 'prompts', 'discret_prompts', 'old_prompts', 'old_discret_prompts', 'feature_names', 'scaler', 'discretizers', 'num_samples', 'num_features', '#classes', '#snr', 'k-top'])

In [34]:
messages = [{
    "role": "user",
    "content": noisy_data['old_prompts'][0]
}]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "low", # **NEW!** Set reasoning effort to low, medium or high
).to(model.device)

inputs['input_ids'].shape

torch.Size([1, 2925])

In [35]:
messages = [{
    "role": "user",
    "content": noisy_data['prompts'][0]
}]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "low", # **NEW!** Set reasoning effort to low, medium or high
).to(model.device)

inputs['input_ids'].shape

torch.Size([1, 1033])

In [36]:
messages = [{
    "role": "user",
    "content": noisy_data['discret_prompts'][0]
}]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "low", # **NEW!** Set reasoning effort to low, medium or high
).to(model.device)

inputs['input_ids'].shape

torch.Size([1, 862])

In [1]:
import json

with open('../src/evaluation/discret_prompts_gemini-2.5-flash_noisySignal_5_4_gemini_responses.json', 'r') as f:
    data = json.load(f)

data[0]


{'filename': 'GMSK_9.09dB__076_20250127_150200.npy',
 'try': 0,
 'prompt': '**ROLE:**\nYou are an expert AI signal classifier specializing in wireless communication modulation schemes.\n\n**OBJECTIVE:**\nYour task is to classify the modulation scheme of a wireless signal based on its provided `KBinsDiscretizer` statistical features. You will use your domain knowledge of signal processing to interpret the features and select the correct classification from a predefined list.\n\n**CONTEXT:**\nThe classification is based on the principle that moments and cumulants of a signal\'s I/Q components create a unique feature set. For instance, the fourth-order cumulant is a powerful differentiator between modulation formats like BPSK, QPSK, and various QAM schemes. The provided statistics are the features extracted from a signal after being processed by a `KBinsDiscretizer`, which quantifies the signal\'s distribution. Your analysis should map these statistical features to the most likely modulat

In [4]:
filenames = [r['filename'] for r in data]
print(len(filenames))
len(set(filenames))

186


186